# Exploring Sentinel-1 radar for Hurricane Harvey Flood Mapping

Motivated by:
Chini, Marco, Ramona Pelich, Luca Pulvirenti, Nazzareno Pierdicca, Renaud Hostache, and Patrick Matgen. “Sentinel-1 InSAR Coherence to Detect Floodwater in Urban Areas: Houston and Hurricane Harvey as A Test Case.” ​Remote Sensing​ 11, no. 2 (January 9, 2019): 107. https://www.mdpi.com/2072-4292/11/2/107.

Relevant background (from above paper):
Category 4 hurricane Harvey’s **first landfall on 25 August 2017** on San Jose Island, Texas. After hitting the Texas mainland on the same day, Harvey weakened to a tropical storm and its speed reduced. As it was only progressing slowly, the amount of rainfall in the region became extremely high and the volume of water took days to drain through rivers, causing widespread catastrophic flooding, most notably in the Houston metropolitan area. The local National Weather Service office in Houston observed **daily rainfall accumulations of 370 mm and 4018 mm on August 26 and 27, respectively**. A total precipitation of 1539 mm in the city of Nederland in southeast Texas, makes Hurricane Harvey the wettest tropical cyclone on record in the United States [43]. As a result, the Houston bayous burst their banks for several days, which led to substantial large-scale flooding, especially to the north and east of the Houston area. The total economic loss from the event amounted to about US$125 billion, with more than 30,000 people evacuated from their homes, more than 17,000 rescues prompted and 106 confirmed deaths in the United States. Hurricane Harvey ties with Hurricane Katrina (2005) as the costliest tropical cyclone on record [44].

Goal: explore flood classification / damaged buildings. Possibly use images from Chini study IW (20m):

    * 18 August 2017 - 24 August 2017 IW VV
    * 24 August 2017 - 30  August 2018 IW HH

In [1]:
# If you get an error here you probably need to authenticate:
#https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/authorize_notebook_server.ipynb
import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl
import shapely
import json
import ee
ee.Initialize()

## Show area and time of interest

In [2]:
gf = gpd.read_file('aoi.geojson')
coordlist = list(gf.geometry[0].exterior.coords)

In [3]:
coordlist

[(-96.9268798828125, 28.502488316130417),
 (-94.12261962890625, 28.502488316130417),
 (-94.12261962890625, 30.0286775329042),
 (-96.9268798828125, 30.0286775329042),
 (-96.9268798828125, 28.502488316130417)]

In [4]:
# ipyleaflet takes (lat,lon coords annoyingly)
def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0])

def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]  

swap_coordinate_xy_for_list(coordlist)

[(28.502488316130417, -96.9268798828125),
 (28.502488316130417, -94.12261962890625),
 (30.0286775329042, -94.12261962890625),
 (30.0286775329042, -96.9268798828125),
 (28.502488316130417, -96.9268798828125)]

In [5]:
p_houston = (-95.3698, 29.7604)
m = Map(center=swap_coordinate_xy_for_location(p_houston), zoom=5)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-08-30")
left_layer = ipyleaflet.TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

polygon = ipyleaflet.Polygon(locations=swap_coordinate_xy_for_list(coordlist), color="red")
m.add_layer(polygon)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Load Sentinel-1 Google Earth Engine Dataset

In [6]:
# Google Earth Engine Tiles!
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    
    return tile_url_template.format(**map_id)

In [7]:
def ReplaceOverlayLayers(map_object, ee_image_object):
    ''' Will update existing map '''
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

In [8]:
#https://developers.google.com/earth-engine/sentinel1
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

In [9]:
# More sophisticated filters
site = ee.Geometry.Polygon(coordlist)
orbitDir = ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
polarization = ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
mode = ee.Filter.eq('instrumentMode', 'IW')
orbit = ee.Filter.eq('relativeOrbitNumber_start', 143) #ASCENDING 34 also worth checking out!
resolution = ee.Filter.eq('resolution_meters', 10)
dates = ee.Filter.date('2017-08-01', '2017-08-31') #landfal August 25. 

collection = (s1
              .filterBounds(site)
              #.filterDate('2016-10-01', '2017-09-30')
              .filter(dates)
              .filter(orbitDir)
              .filter(orbit)
              .filter(mode)
              .filter(polarization)
              .select('VV')
             )

sample_image = collection.first()
              

viz_params = {'bands':'VV', 
              'min': -25, 
              'max': 0, 
              'opacity': 0.6}

In [10]:
# get timestamps of images matching search
def get_timestamps(collection):
    imageList = ee.List(collection.toList(collection.size()))
    timestamps = imageList.map(lambda img: ee.Image(img).date().format())#.format('Y-m-d'))
    return timestamps

In [11]:
def get_ids(collection):
    imageList = ee.List(collection.toList(collection.size()))
    ids = imageList.map(lambda img: ee.Image(img).id())
    return ids

In [12]:
#NOTE that adjacent frames are in this AOI and therefore same collection
# from chini, pre= 2017-08-24T12:22:48  post = 2017-08-30T12:22:03
timestamps = get_timestamps(collection)
ts = timestamps.getInfo()
ts.sort()
ids = get_ids(collection)
ids = ids.getInfo()
ids.sort()
list(zip(ts,ids))

[('2017-08-06T12:22:05',
  'S1A_IW_GRDH_1SDV_20170812T122247_20170812T122317_017890_01E004_81E4'),
 ('2017-08-12T12:22:47',
  'S1A_IW_GRDH_1SDV_20170824T122248_20170824T122318_018065_01E54E_A87C'),
 ('2017-08-18T12:22:05',
  'S1B_IW_GRDH_1SDV_20170806T122205_20170806T122235_006819_00C006_6D40'),
 ('2017-08-24T12:22:48',
  'S1B_IW_GRDH_1SDV_20170818T122205_20170818T122235_006994_00C525_B566'),
 ('2017-08-30T12:22:03',
  'S1B_IW_GRDH_1SDV_20170830T122203_20170830T122232_007169_00CA2C_E7BF'),
 ('2017-08-30T12:22:32',
  'S1B_IW_GRDH_1SDV_20170830T122232_20170830T122257_007169_00CA2C_CC8C')]

## For now, pick two images... 

### before: 2017-08-24T12:22:48
### after:  2017-08-30T12:22:03

In [13]:
pre_image = ee.Image('COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170818T122205_20170818T122235_006994_00C525_B566')
#pre_image.getInfo() all metadata

In [14]:
post_image = ee.Image('COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170830T122203_20170830T122232_007169_00CA2C_E7BF')
#post_image.getInfo()

### Display pre-hurricane image

In [15]:
image_centroid = tuple(site.centroid().getInfo()['coordinates'][::-1])

map1 = ipyleaflet.Map(zoom=8, center=image_centroid, layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)


ee_image_object = pre_image.visualize(**viz_params)
#ee_image_object = post_image.visualize(**viz_params)
map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))
)

polygon = ipyleaflet.Polygon(locations=swap_coordinate_xy_for_list(coordlist), color="red", fill=False)
map1.add_layer(polygon)

map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Display split view (pre-event image on left, post-event on right)

In [16]:
# Split view downtown houston, before flooding on left, after flooding on right
# Simple thresholding to highlight pixels that experienced a big drop in backscatter (open field --> flood scenario)
# In urban areas we might get opposite trend (double bounce causes increase in backscatter)

# NOTE: there still may be orthorectification errors in which case rubber sheeting can be used for fine-scale offsets:
#https://developers.google.com/earth-engine/register

# NOTE: should also probably do some sort of histogram normalization

m = ipyleaflet.Map(zoom=12, center=swap_coordinate_xy_for_location(p_houston), layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)
viz_params = {'bands':'VV', 
              'min': -25, 
              'max': 0, 
              'opacity': 1} # no opacity for this one


#left_layer = ipyleaflet.TileLayer() # OSM
ee_image_object = pre_image.visualize(**viz_params)
left_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))
ee_image_object = post_image.visualize(**viz_params)
right_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Display crude water mask based on backscatter threshold of pre-event image

In [17]:
# Naive standing water mask for single image
# https://developers.google.com/earth-engine/sentinel1
# db>0 --> scattering towards sensor
# db<0 --> away from sensor
# db<-6 --> standing water? (VV according to Schumann Moller 2015)
#mask = image.gte(0) #very bright reflectors

image = pre_image
mask_surface_water = image.lte(-15)
pre_surface_water = image.updateMask(mask_surface_water)

m = ipyleaflet.Map(zoom=12, center=swap_coordinate_xy_for_location(p_houston), layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)
viz_dif = {'bands':'VV', 
            'min': -25, 
            'max': 25, 
            'opacity': 1,
            'palette': ['0000FF','00FFFF'],}

left_layer = ipyleaflet.TileLayer() # OSM
ee_image_object = pre_surface_water.visualize(**viz_dif)
right_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Note that single threshold doesn't work well for images on different dates

In [18]:
# Naive standing water mask for single image
# https://developers.google.com/earth-engine/sentinel1
# db>0 --> scattering towards sensor
# db<0 --> away from sensor
# db<-6 --> standing water (according to Schumann Moller 2015)
#mask = image.gte(0) #very bright reflectors
# Would probably be best to composite temporal stack of pre-event imagery...

image = pre_image
mask_surface_water = image.lte(-15)
pre_surface_water = image.updateMask(mask_surface_water)

image = post_image
mask_surface_water = image.lte(-12)
post_surface_water = image.updateMask(mask_surface_water)

m = ipyleaflet.Map(zoom=12, center=swap_coordinate_xy_for_location(p_houston), layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)
viz_dif = {'bands':'VV', 
            'min': -25, 
            'max': 25, 
            'opacity': 1,
            'palette': ['0000FF','00FFFF'],}

#left_layer = ipyleaflet.TileLayer() # OSM
ee_image_object = pre_surface_water.visualize(**viz_dif)
left_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))

ee_image_object = post_surface_water.visualize(**viz_dif)
right_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Display 'flood map' based on backscatter difference threshold

In [19]:
# Look at a difference image (large decreases in backscatter --> flooded open land, 
#increases in backscatter --> double bounce in areas of trees and urban coridoors
diff_image = post_image.subtract(pre_image)

#decreasers = diff_image.lte(0)
#diff_image = diff_image.updateMask(decreasers)

#increasers = diff_image.gte(0)
increasers = diff_image.gte(8) #number in decibels so orders of magnitude
diff_image = diff_image.updateMask(increasers)

p_kingwood = (-95.18480,30.04940)
m = ipyleaflet.Map(zoom=13, center=swap_coordinate_xy_for_location(p_kingwood), layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)
viz_dif = {'bands':'VV', 
            'min': -10, 
            'max': 10, 
            'opacity': 1,
            #'palette': ['0000FF','00FFFF'], #Cyan high values, blue are low values
            'palette': ['0000FF', 'FFFFFF', 'FF0000'],  #BWR
          } 


left_layer = ipyleaflet.TileLayer() # OSM
ee_image_object = diff_image.visualize(**viz_dif)
right_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [20]:
# Add some GeoJSON from digitial globe - note, this can be slow to render

'''
m = ipyleaflet.Map(zoom=12, center=swap_coordinate_xy_for_location(p_houston), layout=dict(height='500px',width='1000px'))
#dc = ipyleaflet.DrawControl()
#map1.add_control(dc)
viz_params = {'bands':'VV', 
              'min': -25, 
              'max': 0, 
              'opacity': 1} # no opacity for this one


left_layer = ipyleaflet.TileLayer() # OSM
#ee_image_object = pre_image.visualize(**viz_params)
#left_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))
ee_image_object = post_image.visualize(**viz_params)
right_layer = ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))


with open('digitalglobe_crowdsourcing_hurricane_harvey_20170915.geojson', 'r') as f:
    data = json.load(f)

geo_json = ipyleaflet.GeoJSON(data=data) #style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1}
m.add_layer(geo_json)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
#m
'''
print('uncomment to plot, a bit slow to render...')

uncomment to plot, a bit slow to render...
